# Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense
import pprint
from pickle import dump
import tensorflow as tf

In [2]:
songs = pd.read_csv("song_list5.csv")
songs2 = pd.read_csv("mood.csv")

In [3]:
songs.head()

,songid,artist,track,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,5X4Qm0rVLcZeeO4tSDmBg3,Jack Bruce,Running Thro' Our Hands,0.456,0.255,9.0,-15.805,1.0,0.0480,0.94600,0.170000,0.951,0.0532,116.424,253067.0,4.0
1,1pNpt53PZPet9dvJN3RKGr,Prefuse 73,Parachute Panador,0.535,0.806,7.0,-10.289,1.0,0.0642,0.00436,0.019100,0.457,0.3760,90.089,63733.0,4.0
2,3oxz2oCzAWdPzA6In2zA5u,Pasion Vega,La Gata Bajo La Lluvia,0.294,0.482,5.0,-6.406,1.0,0.0430,0.46300,0.000000,0.335,0.2040,166.693,255280.0,3.0
3,05JGVUwt7XJk5FPqH0Wsch,Jonny Lang,Walking Away,0.563,0.631,0.0,-5.144,1.0,0.0324,0.06350,0.000008,0.163,0.5400,115.657,254827.0,4.0
4,3xdgCFMTn6ut8fZYxfAuR0,Skye,All the Promises,0.358,0.611,2.0,-9.752,0.0,0.0454,0.51500,0.000468,0.149,0.1640,171.596,256933.0,4.0


In [4]:
songs2.head()

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,playlist
0,Rajiv Dhall,84,84,3qePCADpvWQ38YCLxXBrdU,0.448,0.413,5,-7.411,1,0.0511,0.0,0.121,0.406,169.319,155729,4,warm_fuzzy_feeling
1,Niall Horan,Put A Little Love On Me,Put A Little Love On Me,1hCTxutVVYvggAQcUUUZAT,0.584,0.465,10,-4.880,1,0.0262,0.0,0.106,0.299,95.969,224586,4,warm_fuzzy_feeling
2,Jake Scott,Favorite T-Shirt (Acoustic),Favorite T-Shirt (Acoustic),0sem2PTbwYJdCHoDn487TE,0.823,0.399,9,-8.608,1,0.0571,0.0,0.123,0.607,114.999,188442,4,warm_fuzzy_feeling
3,Jaymes Young,Happiest Year,Happiest Year,5kcvBnt6DPX3AMEsCx1qDh,0.501,0.167,1,-14.061,1,0.0484,0.0,0.111,0.356,112.275,228443,3,warm_fuzzy_feeling
4,CAL,Dressed Up In White,Dressed Up In White,0MoupiG0cMvbfPV1nwZAry,0.735,0.422,3,-8.208,1,0.0733,0.0,0.208,0.399,146.062,203836,4,warm_fuzzy_feeling


In [5]:
features = songs[[
    "danceability", "energy", "key", "loudness", "mode", "speechiness", 
    "acousticness", "instrumentalness", "liveness", "valence", "tempo", 
    "duration_ms", "time_signature"]].to_numpy()

In [6]:
features[0]

array([ 4.56000e-01,  2.55000e-01,  9.00000e+00, -1.58050e+01,
        1.00000e+00,  4.80000e-02,  9.46000e-01,  1.70000e-01,
        9.51000e-01,  5.32000e-02,  1.16424e+02,  2.53067e+05,
        4.00000e+00])

In [7]:
scaler = StandardScaler()

In [8]:
x_train = scaler.fit_transform(features)
# x_train = features

In [9]:
dump(scaler, open('scaler.pkl', 'wb'))

In [10]:
print(x_train[0])
print(x_train[0].shape)

[-0.47695143 -1.43616823  1.03467011 -1.32564479  0.70711739 -0.34028656
  1.91669133 -0.14321669  3.87500921 -1.70714343 -0.17066299  0.03404049
  0.24368604]
(13,)


In [11]:
# encoder_input = Input(shape=(13,))

# encoder_dense1 = Dense(10, activation='relu')(encoder_input)
# encoder_dense2 = Dense(8, activation='relu')(encoder_dense1)
# encoder_dense3 = Dense(6, activation='relu')(encoder_dense2)
# encoder_output = Dense(4, activation='relu')(encoder_dense3)

# encoder = keras.Model(encoder_input, encoder_output)



# decoder_dense1 = Dense(6, activation='relu')(encoder_output)
# decoder_dense2 = Dense(8, activation='relu')(decoder_dense1)
# decoder_dense3 = Dense(10, activation='relu')(decoder_dense2)
# decoder_output = Dense(13)(decoder_dense3)

# auto_encoder = keras.Model(encoder_input, decoder_output)

# auto_encoder.summary()

In [12]:
# Model Version 2

encoder_input = Input(shape=(13,))

encoder_dense1 = Dense(12, activation='relu')(encoder_input)
encoder_dense2 = Dense(10, activation='relu')(encoder_dense1)
encoder_dense3 = Dense(8, activation='relu')(encoder_dense2)
encoder_dense4 = Dense(6, activation='relu')(encoder_dense3)
encoder_output = Dense(2, activation='relu')(encoder_dense4)

encoder = keras.Model(encoder_input, encoder_output)
encoder.compile()

decoder_dense2 = Dense(6, activation='relu')(encoder_output)
decoder_dense3 = Dense(8, activation='relu')(decoder_dense2)
decoder_dense4 = Dense(10, activation='relu')(decoder_dense3)
decoder_dense5 = Dense(12, activation='relu')(decoder_dense4)
decoder_output = Dense(13)(decoder_dense5)
# decoder_output = Dense(13, activation='softplus')(decoder_dense5)

auto_encoder = keras.Model(encoder_input, decoder_output)

auto_encoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
dense (Dense)                (None, 12)                168       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                130       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 14        
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 18  

In [13]:
auto_encoder.compile(optimizer='nadam',
                    loss='mean_absolute_error',
                    metrics='mean_absolute_percentage_error')

In [14]:
auto_encoder.fit(x_train, x_train,
                epochs=5,
                batch_size=32,
                shuffle=True)

Epoch 1/5
1563/1563 [==============================] - 3s 2ms/step - loss: 0.5844 - mean_absolute_percentage_error: 178.8770
Epoch 2/5
1563/1563 [==============================] - 2s 1ms/step - loss: 0.5127 - mean_absolute_percentage_error: 183.6284
Epoch 3/5
1563/1563 [==============================] - 2s 2ms/step - loss: 0.4802 - mean_absolute_percentage_error: 185.5531
Epoch 4/5
1563/1563 [==============================] - 2s 2ms/step - loss: 0.4688 - mean_absolute_percentage_error: 187.8470
Epoch 5/5
1563/1563 [==============================] - 2s 1ms/step - loss: 0.4630 - mean_absolute_percentage_error: 188.6069


In [15]:
preds = auto_encoder.predict(x_train)

In [16]:
pprint.pprint(preds[0])
pprint.pprint(x_train[0])
print("=======================================================================")
pprint.pprint(preds[1])
pprint.pprint(x_train[1])
print("=======================================================================")
pprint.pprint(preds[2])
pprint.pprint(x_train[2])

array([-1.1171317 , -1.8356392 ,  1.0730016 , -1.479594  ,  0.7007265 ,
       -0.42651197,  1.9171674 , -0.17966518, -0.45103863, -1.2271085 ,
       -0.5789978 , -0.4186229 ,  0.2492581 ], dtype=float32)
array([-0.47695143, -1.43616823,  1.03467011, -1.32564479,  0.70711739,
       -0.34028656,  1.91669133, -0.14321669,  3.87500921, -1.70714343,
       -0.17066299,  0.03404049,  0.24368604])
array([-0.05321953,  0.7792116 ,  0.513899  ,  0.6474677 ,  0.73548454,
       -0.29950082, -0.78845435, -0.6003849 , -0.28535685,  0.18957484,
        0.22055459, -0.13439628,  0.22119108], dtype=float32)
array([-0.03739907,  0.69843612,  0.47428756, -0.17173186,  0.70711739,
       -0.1963104 , -0.87635746, -0.58458172,  1.27879637, -0.49117162,
       -1.06220066, -1.51723037,  0.24368604])
array([ 0.8985118 ,  0.09850469,  0.10858285,  0.23108318,  0.72001404,
       -0.37826705, -0.28191358, -0.62897974, -0.521139  ,  0.7529552 ,
       -0.11813417, -0.13006148,  0.23077178], dtype=float32)


In [17]:
encoder.save('encoder', save_format='h5')

In [21]:
encoder.save('encoder.h5')

In [18]:
# # auto_encoder.save("/nnmodels/auto_encoder")

# filepath = "/nnmodels/auto_encoder"

# tf.keras.models.save_model(
#     auto_encoder, filepath, overwrite=True, include_optimizer=True, save_format=None,
#     signatures=None, options=None
# )

In [19]:
# from tensorflow import keras
# model = keras.models.load_model('path/to/location')